In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [3]:
##load the trained model,scalar pickle file and ohe-hot encoder pickle file
model= load_model('model.keras')

##load the scalar and one-hot encoder
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender=pickle.load(file)

with open('ohe_geo.pkl', 'rb') as file:
    ohe_geo=pickle.load(file)
    
with open('scaler.pkl', 'rb') as file:
    scaler=pickle.load(file)
    


In [5]:
##example input data
input_data = {
    'CreditScore': 700,
    'Geography': 'Germany',
    'Gender': 'Male',
    'Age': 25,
    'Tenure': 5,
    'Balance': 1000.0,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000.0
  
}
    

In [ ]:
geo_encoded = ohe_geo.transform(
    [[input_data['Geography']]]
    ).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [8]:
geo_encoded = ohe_geo.transform(
    pd.DataFrame({'Geography': [input_data['Geography']]})
).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [9]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,700,Germany,Male,25,5,1000.0,2,1,1,50000.0


In [10]:
##encode the categorical features
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,700,Germany,1,25,5,1000.0,2,1,1,50000.0


In [11]:
##concatination one hot encoded data
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,1,25,5,1000.0,2,1,1,50000.0,0.0,1.0,0.0


In [12]:
##scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[ 5.01787943e-01,  9.13247552e-01, -1.32129295e+00,
        -1.34471949e-03, -1.20245957e+00,  8.08436154e-01,
         6.49202671e-01,  9.74816989e-01, -8.76832208e-01,
        -9.98501123e-01,  1.72572313e+00, -5.76388018e-01]])

In [18]:
##predicting the output
predictions = model.predict(input_scaled)
print(predictions)

# Convert predictions to class labels
pp=predictions[0][0]  # Assuming binary classification, get the first prediction
if pp > 0.5:
    print("The customer is likely to leave the bank.")
else:
    print("The customer is likely to stay with the bank.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
[[0.01533656]]
The customer is likely to stay with the bank.
